# Fix RAPIDS Installation


In [ ]:
import pandas as pd 
test = pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv")
print("Test shape",test.shape)
display(test.head())

if len(test) <10:
    prunt()

DEBUG = False
v_debug = 100

In [ ]:
%%time
!pip install --find-links /kaggle/input/downgrade-pandas /kaggle/input/downgrade-pandas/pandas-1.5.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
# BIBLIOTECAS INSTALAÇÂO
!pip install /kaggle/input/pre-processamento-treino/en_core_web_md-3.7.1-py3-none-any.whl
!pip install /kaggle/input/lexical-diversity/lexical_diversity-0.1.1-py3-none-any.whl
!pip install /kaggle/input/lexical-diversity/syllapy-0.7.2-py3-none-any.whl

import sys
sys.path.append('/kaggle/input/aes-fucoes')
from funcoes import *

# Load Libraries and Data

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

import numpy as np, gc, re 
import pandas as pd 

train = pd.read_csv("/kaggle/input/pre-processamento-treino/train_prompt_fold.csv")
if DEBUG:
    train = train[:v_debug]
print("Train shape",train.shape)
display(train.head())
print()

test = pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv")
print("Test shape",test.shape)
display(test.head())


# Stratified 15 K Fold

In [ ]:
# from sklearn.model_selection import StratifiedKFold

# FOLDS = 15
# train["fold"] = -1
# skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)
# for fold,(train_index, val_index) in enumerate(skf.split(train,train["score"])):
#     train.loc[val_index,"fold"] = fold
# print('Train samples per fold:')
# train.fold.value_counts().sort_index()

In [ ]:
# promtp = pd.read_csv('/kaggle/input/funcoes-featurestext/predicted_prompt.csv')
# # Add prompt_name to the train DataFrame by merging on essay_id
# train = pd.merge(train, promtp[['essay_id', 'prompt_name']], on='essay_id', how='left')
# train

# from sklearn.model_selection import StratifiedGroupKFold

# # Number of folds
# FOLDS = 3

# # Initialize the 'fold' column with -1
# train["fold"] = -1

# # Initialize the StratifiedGroupKFold object
# sgkf = StratifiedGroupKFold(n_splits=FOLDS, shuffle=True, random_state=42)

# # Assuming 'group' is the column used for grouping
# for fold, (train_index, val_index) in enumerate(sgkf.split(train, train["score"], groups=train["prompt_name"])):
#     train.loc[val_index, "fold"] = fold

# # Display the number of samples per fold
# print('Train samples per fold:')
# print(train.fold.value_counts().sort_index())

# Generate Embeddings

In [ ]:
from transformers import AutoModel,AutoTokenizer
import torch, torch.nn.functional as F
from tqdm import tqdm

In [ ]:
def mean_pooling_ultima(model_output, attention_mask):
    # ULTIMA SAIDA
    token_embeddings = model_output.last_hidden_state.detach().cpu()
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(
        input_mask_expanded.sum(1), min=1e-9
    )

def mean_pooling_media_todas(model_output, attention_mask):
    # MEDIA TODAS CAMADAS
    token_embeddings = model_output.hidden_states[:]
    
    # Calcula a média das três primeiras camadas
    token_embeddings = torch.stack(token_embeddings).mean(dim=0)
    
    # Detach e move para a CPU
    token_embeddings = token_embeddings.detach().cpu()
    
    # Expandir a máscara de atenção para a dimensão do token_embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    
    # Aplicar a máscara de atenção aos embeddings
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    
    # Calcular a média levando em conta a máscara de atenção
    return sum_embeddings / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# def mean_pooling(model_output, attention_mask):
#      # CONCATENA AS 3 PRIMEIRAS
#     token_embeddings = torch.cat(model_output.hidden_states[:3], dim=-1).detach().cpu()
    
#     # Expandir a máscara de atenção para a dimensão do token_embeddings
#     input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    
#     # Aplicar a máscara de atenção aos embeddings
#     sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    
#     # Calcular a média levando em conta a máscara de atenção
#     return sum_embeddings / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def mean_pooling_3_primeiras_media(model_output, attention_mask):
    # Extrai as três primeiras camadas
    token_embeddings = model_output.hidden_states[:3]
    
    # Calcula a média das três primeiras camadas
    token_embeddings = torch.stack(token_embeddings).mean(dim=0)
    
    # Detach e move para a CPU
    token_embeddings = token_embeddings.detach().cpu()
    
    # Expandir a máscara de atenção para a dimensão do token_embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    
    # Aplicar a máscara de atenção aos embeddings
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    
    # Calcular a média levando em conta a máscara de atenção
    return sum_embeddings / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# def mean_pooling(model_output, attention_mask):
#     # Extrai as primeiras 8 camadas e as últimas 4 camadas
#     first_8_layers = model_output.hidden_states[:8]
#     last_4_layers = model_output.hidden_states[-4:]
    
#     # Calcula a média das 8 primeiras camadas
#     first_8_mean = torch.stack(first_8_layers).mean(dim=0)
    
#     # Calcula a média das 4 últimas camadas
#     last_4_mean = torch.stack(last_4_layers).mean(dim=0)
    
#     # Detach e move para a CPU
#     first_8_mean = first_8_mean.detach().cpu()
#     last_4_mean = last_4_mean.detach().cpu()
    
#     # Expandir a máscara de atenção para a dimensão dos embeddings
#     input_mask_expanded = attention_mask.unsqueeze(-1).expand(first_8_mean.size()).float()
    
#     # Aplicar a máscara de atenção aos embeddings das 8 primeiras camadas
#     sum_first_8_embeddings = torch.sum(first_8_mean * input_mask_expanded, 1)
    
#     # Aplicar a máscara de atenção aos embeddings das 4 últimas camadas
#     sum_last_4_embeddings = torch.sum(last_4_mean * input_mask_expanded, 1)
    
#     # Calcular a média levando em conta a máscara de atenção
#     first_8_mean_pooled = sum_first_8_embeddings / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
#     last_4_mean_pooled = sum_last_4_embeddings / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    
#     # Concatenar as médias das 8 primeiras camadas e das 4 últimas camadas
#     concatenated_mean = torch.cat((first_8_mean_pooled, last_4_mean_pooled), dim=-1)
    
#     return concatenated_mean

# def mean_pooling(model_output, attention_mask):
#     # Extrai as primeiras 8 camadas e as últimas 4 camadas
#     first_8_layers = model_output.hidden_states[:4]
#     last_4_layers = model_output.hidden_states[-8:-4]
    
#     # Calcula a média das 8 primeiras camadas
#     first_8_mean = torch.stack(first_8_layers).mean(dim=0)
    
#     # Calcula a média das 4 últimas camadas
#     last_4_mean = torch.stack(last_4_layers).mean(dim=0)
    
#     # Detach e move para a CPU
#     first_8_mean = first_8_mean.detach().cpu()
#     last_4_mean = last_4_mean.detach().cpu()
    
#     # Expandir a máscara de atenção para a dimensão dos embeddings
#     input_mask_expanded = attention_mask.unsqueeze(-1).expand(first_8_mean.size()).float()
    
#     # Aplicar a máscara de atenção aos embeddings das 8 primeiras camadas
#     sum_first_8_embeddings = torch.sum(first_8_mean * input_mask_expanded, 1)
    
#     # Aplicar a máscara de atenção aos embeddings das 4 últimas camadas
#     sum_last_4_embeddings = torch.sum(last_4_mean * input_mask_expanded, 1)
    
#     # Calcular a média levando em conta a máscara de atenção
#     first_8_mean_pooled = sum_first_8_embeddings / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
#     last_4_mean_pooled = sum_last_4_embeddings / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    
#     # Concatenar as médias das 8 primeiras camadas e das 4 últimas camadas
#     concatenated_mean = torch.cat((first_8_mean_pooled, last_4_mean_pooled), dim=-1)
    
#     return concatenated_mean

def mean_pooling_4_meio(model_output, attention_mask):
    # SOMENTE AS 4 DO MEIO
    last_4_layers = model_output.hidden_states[-8:-4]

    # Calcula a média das 4 últimas camadas
    last_4_mean = torch.stack(last_4_layers).mean(dim=0)
    last_4_mean = last_4_mean.detach().cpu()

    # Expandir a máscara de atenção para a dimensão dos embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_4_mean.size()).float()

    # Aplicar a máscara de atenção aos embeddings das 4 últimas camadas
    sum_last_4_embeddings = torch.sum(last_4_mean * input_mask_expanded, 1)

    # Calcular a média levando em conta a máscara de atenção
    last_4_mean_pooled = sum_last_4_embeddings / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    return last_4_mean_pooled

In [ ]:
class EmbedDataset(torch.utils.data.Dataset):
    def __init__(self,df,tokenizer,max_length):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max = max_length
    def __len__(self):
        return len(self.df)
    def __getitem__(self,idx):
        text = self.df.loc[idx,"full_text"]
        tokens = self.tokenizer(
                text,
                None,
                add_special_tokens=True,
                padding='max_length',
                truncation=True,
                max_length=self.max,
                return_tensors="pt")
        tokens = {k:v.squeeze(0) for k,v in tokens.items()}
        return tokens

# Extract Embeddings

In [ ]:
def get_embeddings(disk_name='', max_length=1024, batch_size=32, mean_pooling=None, compute_train=True, compute_test=True, tokenizerpath=None):

    global train, test

    DEVICE = "cuda:1" # EXTRACT EMBEDDINGS WITH GPU #2
#     path = "/kaggle/input/download-huggingface-models/"
#     path = "/kaggle/input/modeloslearningfinal/deberta-v3-large-2-stagios-asap-Fold0/"
#     disk_name = path + model_name
#     disk_name = path + model_name.replace("/","_")
#     model = AutoModel.from_pretrained( disk_name , trust_remote_code=True)
    model = AutoModel.from_pretrained(disk_name, output_hidden_states=True, trust_remote_code=True)
    if tokenizerpath is None:
        tokenizer = AutoTokenizer.from_pretrained( disk_name , trust_remote_code=True)
    else:
        tokenizer = AutoTokenizer.from_pretrained( '/kaggle/input/download-huggingface-models/microsoft_deberta-v3-large/' , trust_remote_code=True)

    ds_tr = EmbedDataset(train, tokenizer, max_length)
    embed_dataloader_tr = torch.utils.data.DataLoader(ds_tr,
                            batch_size=batch_size,
                            shuffle=False)
    ds_te = EmbedDataset(test, tokenizer, max_length)
    embed_dataloader_te = torch.utils.data.DataLoader(ds_te,
                            batch_size=batch_size,
                            shuffle=False)
    
    model = model.to(DEVICE)
    model.eval()

    # COMPUTE TRAIN EMBEDDINGS
    all_train_text_feats = []
    if compute_train:
        for batch in tqdm(embed_dataloader_tr,total=len(embed_dataloader_tr)):
            input_ids = batch["input_ids"].to(DEVICE)
            attention_mask = batch["attention_mask"].to(DEVICE)
            with torch.no_grad():
                with torch.cuda.amp.autocast(enabled=True):
                    model_output = model(input_ids=input_ids,attention_mask=attention_mask)
            sentence_embeddings = mean_pooling(model_output, attention_mask.detach().cpu())
            # Normalize the embeddings
            sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
            sentence_embeddings =  sentence_embeddings.squeeze(0).detach().cpu().numpy()
            all_train_text_feats.extend(sentence_embeddings)
    all_train_text_feats = np.array(all_train_text_feats)

    # COMPUTE TEST EMBEDDINGS
    all_test_text_feats = []
    if compute_test:
        for batch in embed_dataloader_te:
            input_ids = batch["input_ids"].to(DEVICE)
            attention_mask = batch["attention_mask"].to(DEVICE)
            with torch.no_grad():
                with torch.cuda.amp.autocast(enabled=True):
                    model_output = model(input_ids=input_ids,attention_mask=attention_mask)
            sentence_embeddings = mean_pooling(model_output, attention_mask.detach().cpu())
            # Normalize the embeddings
            sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
            sentence_embeddings =  sentence_embeddings.squeeze(0).detach().cpu().numpy()
            all_test_text_feats.extend(sentence_embeddings)
        all_test_text_feats = np.array(all_test_text_feats)
    all_test_text_feats = np.array(all_test_text_feats)

    # CLEAR MEMORY
    del ds_tr, ds_te
    del embed_dataloader_tr, embed_dataloader_te
    del model, tokenizer
    del model_output, sentence_embeddings, input_ids, attention_mask
    gc.collect()
    torch.cuda.empty_cache()

    # RETURN EMBEDDINGS
    return all_train_text_feats, all_test_text_feats

In [ ]:
# EMBEDDINGS TO LOAD/COMPUTE
# PARAMETERS = (MODEL_NAME, MAX_LENGTH, BATCH_SIZE)
# CHOOSE LARGEST BATCH SIZE WITHOUT MEMORY ERROR

models = [
#     ('microsoft/deberta-base', 1024, 32),
#     ('microsoft/deberta-large', 1024, 8),
#     ('microsoft/deberta-v3-large', 1024, 8),
#     ('allenai/longformer-base-4096', 1024, 32),
#     ('google/bigbird-roberta-base', 1024, 32),
#     ('google/bigbird-roberta-large', 1024, 8),
    
#     ('/kaggle/input/modeloslearningfinal/deberta-v3-large-2-stagios-asap-Fold0/checkpoint-2400/', 1024, 8, mean_pooling_4_meio, 'asap_4_meio'),
#     ('/kaggle/input/modeloslearningfinal/deberta-v3-large-2-stagios-asap-Fold0/checkpoint-2400/', 1024, 8, mean_pooling_3_primeiras_media, 'asap_3_primeiras_media', 1),
    ('/kaggle/input/download-huggingface-models/microsoft_deberta-v3-large/', 1024, 8, mean_pooling_media_todas, 'deberta_media_todas', None),
#     ('/kaggle/input/download-huggingface-models/microsoft_deberta-v3-large/', 1024, 8, mean_pooling_ultima, 'deberta_ultima', None),
    ('/kaggle/input/download-huggingface-models/google_bigbird-roberta-large/', 1024, 8, mean_pooling_4_meio, 'bigbird_roberta_4_meio', None),
#     ('/kaggle/input/download-huggingface-models/allenai_longformer-base-4096/', 1024, 32, mean_pooling_ultima, 'allenai_longformer_4_ultima', None),
]

In [ ]:
%%time
path = "/kaggle/input/funcoes-featurestext/"
all_train_embeds = []
all_test_embeds = []

for (model, max_length, batch_size, mean_func_, name_, tk_) in models:
    name = path +name_+ '.npy'
    if os.path.exists(name):
        _, test_embed = get_embeddings(disk_name=model, max_length=max_length, batch_size=batch_size, mean_pooling=mean_func_, compute_train=False, tokenizerpath=tk_)
        train_embed = np.load(name)
        print(f"Loading train embeddings for {name} {train_embed.shape}")
        if DEBUG:
            train_embed = train_embed[:v_debug]
            print(f"DEBUG Loading train embeddings for {name} {train_embed.shape}")
    else:
        print(f"Computing train embeddings for {name}")
        train_embed, test_embed = get_embeddings(disk_name=model, max_length=max_length, batch_size=batch_size, mean_pooling=mean_func_, compute_train=True, tokenizerpath=tk_)
        np.save(name_+ '.npy', train_embed)
    all_train_embeds.append(train_embed)
    all_test_embeds.append(test_embed)

del train_embed, test_embed



# Deberta inf

In [ ]:
import pandas as pd 
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    Trainer, 
    TrainingArguments, 
    DataCollatorWithPadding
)
from datasets import Dataset
from glob import glob
import gc
import torch
from scipy.special import softmax

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace


def executa_modelo_transform(df_, path_model, max_lenght, eval_batch, path_tokenizer=None):
    
    models = glob(path_model)
    print(models)
    
    if path_tokenizer is None:
        path_tokenizer = models[0]
        
    tokenizer = AutoTokenizer.from_pretrained(path_tokenizer)

    def tokenize(sample):
        return tokenizer(sample['full_text'], max_length=max_lenght, truncation=True)

    
    ds = Dataset.from_pandas(df_).map(tokenize).remove_columns(['essay_id', 'full_text'])

    args = TrainingArguments(
        ".", 
        per_device_eval_batch_size=eval_batch, 
        report_to="none"
    )


    predictions = []

    for model in models:
        model = AutoModelForSequenceClassification.from_pretrained(model)
        trainer = Trainer(
            model=model, 
            args=args, 
            data_collator=DataCollatorWithPadding(tokenizer), 
            tokenizer=tokenizer
        )

        preds = trainer.predict(ds).predictions
        predictions.append(softmax(preds, axis=-1))
#         predictions.append(preds)
        del model, trainer
        torch.cuda.empty_cache()
        gc.collect()
        
    predicted_score = 0.

    for p in predictions:
        predicted_score += p

    predicted_score /= len(predictions)
    
    return predicted_score

def executa_modelo_transform2(df_, path_model, max_lenght, eval_batch, path_tokenizer=None):
    
    models = [os.path.join(path_model, d) for d in os.listdir(path_model) if os.path.isdir(os.path.join(path_model, d))]
    print(models)
    
    if path_tokenizer is None:
        path_tokenizer = models[0]
        
    tokenizer = AutoTokenizer.from_pretrained(path_tokenizer)

    def tokenize(sample):
        return tokenizer(sample['full_text'], max_length=max_lenght, truncation=True)

    
    ds = Dataset.from_pandas(df_).map(tokenize).remove_columns(['essay_id', 'full_text'])

    args = TrainingArguments(
        ".", 
        per_device_eval_batch_size=eval_batch, 
        report_to="none"
    )


    predictions = []
    predictionsarg = []

    for model in models:
        model = AutoModelForSequenceClassification.from_pretrained(model)
        trainer = Trainer(
            model=model, 
            args=args, 
            data_collator=DataCollatorWithPadding(tokenizer), 
            tokenizer=tokenizer
        )

        preds = trainer.predict(ds)
        predictionsarg.append(preds.predictions.argmax(-1))
        predictions.append(preds.predictions)
        del model, trainer
        torch.cuda.empty_cache()
        gc.collect()
        
    predicted_score = 0.

    for p in predictionsarg:
        predicted_score += p

    predicted_score /= len(predictionsarg)
    
    predictions = np.array(predictions)
    logits = predictions.mean(axis=0)
    
    return predicted_score, logits


In [ ]:
def convert_to_array(logits_str):
    try:
        # Remove brackets and split the string into individual numbers
        logits_list = logits_str.strip('[]').split()
        # Convert the list of strings to a NumPy array of floats
        return np.array(logits_list, dtype=float)
    except (ValueError, SyntaxError):
        return np.array([])

In [ ]:
# predicted_train = executa_modelo_transform(train, 
#                                             '/kaggle/input/modeloslearningfinal/deberta-v3-large-2-stagios-asap-Fold0/checkpoint-2400/', 
#                                             1024, 
#                                             1,
#                                             path_tokenizer = '/kaggle/input/deberta-large-fold0/deberta-large-fold0 (1)/deberta-large-fold0/checkpoint-1200',
#                                            )
# np.save('predicted_train_deb_asap.npy', predicted_train)
predicted_train = pd.read_csv('/kaggle/input/funcoes-featurestext/deberta-v3-large-groupfold/oof_predictions.csv')
sorted_predicted_train = pd.merge(train[['essay_id']], predicted_train, on='essay_id', how='left')
predicted_test = executa_modelo_transform2(test, 
                                            '/kaggle/input/funcoes-featurestext/deberta-v3-large-groupfold/', 
                                            1024, 
                                            1,
                                            path_tokenizer = '/kaggle/input/deberta-large-fold0/deberta-large-fold0 (1)/deberta-large-fold0/checkpoint-1200',
                                           )

# if DEBUG:
#     predicted_train = predicted_train[:v_debug]

# display(predicted_train)

predicted_train = sorted_predicted_train['prediction']+1
predicted_test = predicted_test[0] +1

display(predicted_train)
display(predicted_test)

In [ ]:
predicted_train.shape, predicted_test.shape

In [ ]:
# predicted_train_2d = predicted_train[:, np.newaxis]
# predicted_test_2d = predicted_test[:, np.newaxis]

# display(predicted_train_2d)
# predicted_train_2d.shape, predicted_test_2d.shape

# Combine Feature Embeddings

In [ ]:
%%time
# CRIAÇÂO DAS FEATURES --------------------------------
CRIA_FEATURES_TRAIN = False
CRIA_ORDER_DICT = False
if CRIA_FEATURES_TRAIN:
    if CRIA_ORDER_DICT:
        docs_dict = doc_dict(train)
        save_docs_dict(docs_dict, "documentos_dict.spacy")
        print(len(docs_dict))
    else:
        # Carregar o dicionário de documentos
        %time
        docs_dict = load_docs_dict("/kaggle/input/pre-processamento-treino/documentos_dict.spacy",
                                   nlp)

    ordered_docs = [docs_dict[str(essay_id)] for essay_id in train['essay_id']]
    ordered_docs = pd.Series(ordered_docs)

    docs_features = []
    for doc in tqdm(ordered_docs[:], desc="Processando documentos"):
        docs_features.append(process_document(doc))

    # Convertendo a lista de features em um DataFrame
    df = pd.DataFrame(docs_features)
#     pd.set_option('display.max_columns', None)
    display(df)
    display(df.shape)
    df.to_csv("new_features.csv",index=None)
else:
    
    df = pd.read_csv('/kaggle/input/pre-processamento-treino/train_features.csv')
#     pd.set_option('display.max_columns', None)
    if DEBUG:
        df = df[:v_debug]
    display(df)
    display(df.shape)

In [ ]:
# CRIAÇÂO FEATURES TESTE --------------------------------
docs_dicttest = doc_dict(test)
ordered_docstest = [docs_dicttest[str(essay_id)] for essay_id in test['essay_id']]
ordered_docstest = pd.Series(ordered_docstest)
ordered_docstest
docs_featurestest = []
for doc in tqdm(ordered_docstest, desc="Processando documentos"):
    docs_featurestest.append(process_document(doc))
    
# Convertendo a lista de features em um DataFrame
dftest = pd.DataFrame(docs_featurestest)
dftest = dftest
dftest

In [ ]:
def filter_overfitting_features(df_):
    print(df_.shape)
    remove_overfiting_features = ['inf_word_taassc',
#                                   "overlap_coehsion_", "syntactic_density", "sim_total", "tamanho_texto", "fea_letras_", "lex_rep_", 
#                                   "clause_score_", "Rare_Percentage_", "long_word_", "spelling_errors_", "Readability_", '_per_sentence'
                                 ]

#     filtered_columns = [col for col in df_.columns if not any(col.startswith(prefix) for prefix in remove_overfiting_features)]
    filtered_columns = [col for col in df_.columns if not any(prefix in col for prefix in remove_overfiting_features)]
    return df[filtered_columns]
df = filter_overfitting_features(df)
# pd.set_option('display.max_columns', None)
print(df.shape)
feature_names = df.columns.to_list()

fill na

In [ ]:
# all_train_filled = df[feature_names].fillna(0).to_numpy()
# all_test_filled = dftest[feature_names].fillna(0).to_numpy()

mean_values = df[feature_names].median()
all_train_filled = df[feature_names].fillna(mean_values).to_numpy()
all_test_filled = dftest[feature_names].fillna(mean_values).to_numpy()

all_train_ = df[feature_names].to_numpy()
all_test_ = dftest[feature_names].to_numpy()

all_train_.shape, all_test_.shape

In [ ]:
# try:
#     all_train_embeds = np.array(all_train_embeds).squeeze()
#     all_test_embeds = np.array(all_test_embeds).squeeze()
# except:
#     print("concatena")
all_train_embeds = np.concatenate(all_train_embeds, axis=1)
all_test_embeds = np.concatenate(all_test_embeds, axis=1)

if DEBUG:
    all_train_embeds = all_train_embeds[:v_debug]

all_train_embeds.shape, all_test_embeds.shape

In [ ]:
# Concatenação ao longo do eixo 1 (colunas)
all_train_embeds_filled = np.concatenate([all_train_embeds, all_train_filled
                                  ], axis=1)
all_test_embeds_filled = np.concatenate([all_test_embeds, all_test_filled, 
                                 ], axis=1)
all_train_embeds = np.concatenate([all_train_embeds, all_train_
                                  ], axis=1)
all_test_embeds = np.concatenate([all_test_embeds, all_test_, 
                                 ], axis=1)

np.save('all_train_embeds_filled.npy', all_train_embeds_filled)
np.save('all_train_embeds.npy', all_train_embeds)

gc.collect()
print('Our concatenated train embeddings have shape', all_train_embeds.shape )
display(all_train_embeds[:10])

normalização

In [ ]:
from sklearn.preprocessing import StandardScaler

# Criar o objeto StandardScaler
scaler = StandardScaler()

# Fit nos dados de treino e transformá-lo
all_train_embeds_scaled = scaler.fit_transform(all_train_embeds_filled)

# Aplicar a mesma transformação nos dados de teste
all_test_embeds_scaled = scaler.transform(all_test_embeds_filled)

# Train RAPIDS cuML SVR
Documentation for RAPIDS SVR is [here][1]. Using RAPIDS support vector regression (SVR) is a great model when we have 1000s of features because it is quick and it naturally will perform feature selection and remove features to prevent overfitting.

Note that SVR model likes the inputs to be standardized so that each feature is rougly mean 0 and std 1. We approximate this by performing L2 norm on the extracted embeddings.

[1]: https://docs.rapids.ai/api/cuml/stable/api.html#support-vector-machines

In [ ]:
from cuml.svm import SVR
import cuml
print('RAPIDS version',cuml.__version__)

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from lightgbm import log_evaluation, early_stopping
import lightgbm as lgb
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score
import matplotlib.pyplot as plt

def quadratic_weighted_kappa(y_true, y_pred):
    y_true = (y_true + a).clip(1, 6).round()
    y_pred = (y_pred + a).clip(1, 6).round()
    qwk = cohen_kappa_score(y_true, y_pred, weights="quadratic")
    return 'QWK', qwk, True
def qwk_obj(y_true, y_pred):
    labels = y_true + a
    preds = y_pred + a
    preds = preds.clip(1, 6)
    f = 1/2*np.sum((preds-labels)**2)
    g = 1/2*np.sum((preds-a)**2+b)
    df = preds - labels
    dg = preds - a
    grad = (df/g - f*dg/g**2)*len(labels)
    hess = np.ones(len(labels))
    return grad, hess
a = 0
b = 0

# Função callback para ajustar o learning rate de acordo com intervalos específicos
def adjust_learning_rate(current_round):
    if current_round < 500:
        return 0.1
    elif current_round < 1000:
        return 0.05
    elif current_round < 1500:
        return 0.025
    else:
        return 0.01

# Callbacks para o treinamento do modelo
callbacks = [
    log_evaluation(period=50),
    early_stopping(stopping_rounds=200, first_metric_only=True, verbose=-1),
#     lgb.reset_parameter(learning_rate=lambda iter: adjust_learning_rate(iter)),
]

In [ ]:
def selecionar_features_importantes(lista_modelos, dados_treino, maior_que_x):
    # Se a lista de modelos estiver vazia, considerar todas as features como importantes
    if not lista_modelos and maior_que_x==0:
        print("Lista de modelos vazia, retornando todas as features.")
        return np.ones(dados_treino.shape[1], dtype=bool)
    elif not lista_modelos and maior_que_x>0:
        print("Lendo modelo LGBM salvo")
        data = pd.read_pickle('/kaggle/input/pre-processamento-treino/RESULT_TRAIN.pkl')
        lista_modelos = data[0]['models']
    
    # Inicializar um array para armazenar a importância das features para cada modelo
    importancias_features = []
    
    # Calcular a importância das features para cada modelo
    for modelo in lista_modelos:
        importancia_atual = modelo.feature_importances_
        print("Importância das features para um modelo:", importancia_atual[:50])
        importancias_features.append(importancia_atual)
    
    # Converter a lista de importâncias das features em um array numpy
    importancias_features = np.array(importancias_features)
    print("Array de importâncias das features:", importancias_features[:50])

    # Calcular a média das importâncias das features
    media_importancias = np.mean(importancias_features, axis=0)
    print("Média das importâncias das features:", media_importancias[:50])
    
    # Verificar quais features têm média de importância maior que um limiar, por exemplo 0.01
    features_selecionadas = media_importancias > maior_que_x
    print("Features selecionadas (índice booleano):", features_selecionadas[:50])
    
    # Selecionar as features do conjunto de dados de treinamento
    dados_treino_selecionados = dados_treino[:, features_selecionadas]
    print("Dados de treino selecionados com base nas features importantes:", dados_treino_selecionados.shape)
    
    return features_selecionadas

from sklearn.feature_selection import RFE

def selecionar_features_importantes_linear(lista_modelos, dados_treino, dados_alvo, n_features_to_select=2000):
    # Suponha que todos os modelos em lista_modelos são SVR e vamos usar LinearSVR para RFE
    importancias_features = []
    
    # Usar RFE para encontrar as features mais importantes
    for modelo in lista_modelos:
        # Usar LinearSVR como um aproximador para o SVR original
        aproximador = SVR(c=10)
        selector = RFE(aproximador, n_features_to_select=n_features_to_select, step=1)
        selector = selector.fit(dados_treino, dados_alvo)
        importancias_features.append(selector.support_)
        print("Features selecionadas para um modelo:", selector.support_)
    
    # Média das seleções de features (booleana, portanto usa moda)
    features_selecionadas = np.mean(importancias_features, axis=0) > 5  # Considera uma feature importante se foi selecionada pela maioria dos modelos
    print("Features finais selecionadas:", features_selecionadas)
    
    # Selecionar as features do conjunto de dados de treinamento
    dados_treino_selecionados = dados_treino[:, features_selecionadas]
    print("Dados de treino selecionados com base nas features importantes:", dados_treino_selecionados.shape)
    
    return features_selecionadas


In [ ]:
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim as optim
import copy
class SimpleNN(nn.Module):
    def __init__(self, input_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x
class DeepNN(nn.Module):
    def __init__(self, input_size):
        super(DeepNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 16)
        self.fc5 = nn.Linear(16, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)
        return x
class WideNN(nn.Module):
    def __init__(self, input_size):
        super(WideNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class DeepNN2(nn.Module):
    def __init__(self, input_size, layer_sizes=[128, 64, 32, 16], dropout_rate=0.2, use_batch_norm=True):
        super(DeepNN2, self).__init__()
        self.layers = nn.ModuleList()
        self.use_batch_norm = use_batch_norm

        # Iterar sobre as camadas e adicionar linear, batch norm (opcional) e dropout
        previous_size = input_size
        for size in layer_sizes:
            self.layers.append(nn.Linear(previous_size, size))
            if use_batch_norm:
                self.layers.append(nn.BatchNorm1d(size))
            
            self.layers.append(nn.Dropout(dropout_rate))
            previous_size = size

        # Última camada, sem batch norm ou dropout
        self.output_layer = nn.Linear(layer_sizes[-1], 1)

    def forward(self, x):
        for layer in self.layers:
            if isinstance(layer, nn.Linear):
                x = torch.relu(layer(x))
            else:
                x = layer(x)
        x = self.output_layer(x)
        return x
    
class DeepNN3(nn.Module):
    def __init__(self, input_size, layer_sizes=[128, 64, 32, 16], dropout_rate=0.2, use_batch_norm=True, activation_func=torch.relu):
        super(DeepNN3, self).__init__()
        self.layers = nn.ModuleList()
        self.use_batch_norm = use_batch_norm
        self.activation_func = activation_func

        # Iterar sobre as camadas e adicionar linear, batch norm (opcional) e dropout
        previous_size = input_size
        for size in layer_sizes:
            self.layers.append(nn.Linear(previous_size, size))
            if use_batch_norm:
                self.layers.append(nn.BatchNorm1d(size))
            self.layers.append(nn.Dropout(dropout_rate))
            previous_size = size

        # Última camada, sem batch norm ou dropout
        self.output_layer = nn.Linear(layer_sizes[-1], 1)

    def forward(self, x):
        for layer in self.layers:
            if isinstance(layer, nn.Linear):
                x = self.activation_func(layer(x))
            else:
                x = layer(x)
        x = self.output_layer(x)
        return x
class FlexibleBatchNN(nn.Module):
    def __init__(self, input_size, layer_sizes=[128, 64, 32, 16], dropout_rate=0.2, use_batch_norm=True):
        super(FlexibleBatchNN, self).__init__()
        self.use_batch_norm = use_batch_norm
        self.layers = nn.ModuleList()
        
        previous_size = input_size
        for size in layer_sizes:
            self.layers.append(nn.Linear(previous_size, size))
            if use_batch_norm:
                self.layers.append(nn.BatchNorm1d(size))
            self.layers.append(nn.Dropout(dropout_rate))
            previous_size = size
        
        self.output_layer = nn.Linear(layer_sizes[-1], 1)

    def forward(self, x):
        for layer in self.layers:
            if isinstance(layer, nn.BatchNorm1d) and x.shape[0] == 1:
                x = layer(x)  # aplicando BN mesmo com batch size de 1
            elif not isinstance(layer, nn.BatchNorm1d):
                x = torch.relu(layer(x))
        x = self.output_layer(x)
        return x
from transformers import BertModel, BertTokenizer


    
def train_model_with_early_stopping(model, criterion, optimizer, X_train, y_train, X_valid, y_valid, num_epochs=10, batch_size=32, n_epochs_stop=5, lr_decay_step=2, lr_decay_gamma=0.5):
    best_loss = float('inf')
    epochs_no_improve = 0
    best_model = None
    
    # Definir o scheduler para decair o learning rate
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=lr_decay_step, gamma=lr_decay_gamma)
    
    # Verificar se a GPU está disponível e mover o modelo para a GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    for epoch in range(num_epochs):
        model.train()
        for i in range(0, len(X_train), batch_size):
            if len(X_train) - i < 2 and len(X_train) - i > 0:  # Check if the remaining samples are less than 2
                continue  # Skip this batch if less than 2 samples remaining
            inputs = torch.tensor(X_train[i:i+batch_size]).float().to(device)
            targets = torch.tensor(y_train[i:i+batch_size]).float().to(device)
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        # Aplicar o scheduler após cada época e registrar quando o learning rate muda
        last_lr = scheduler.get_last_lr()[0]  # Guarda o último learning rate antes de atualizar
        scheduler.step()
        current_lr = scheduler.get_last_lr()[0]  # Pega o novo learning rate após atualizar

        # Verifica se o learning rate mudou nesta época
        if current_lr != last_lr:
            print(f"Learning rate updated from {last_lr} to {current_lr} at epoch {epoch + 1}")
        
        # Validação no final de cada época
        model.eval()
        with torch.no_grad():
            valid_loss = 0
            for j in range(0, len(X_valid), batch_size):
                if len(X_valid) - j < 2 and len(X_valid) - j > 0:  # Same check for validation data
                    continue  # Skip this batch
                inputs = torch.tensor(X_valid[j:j+batch_size]).float().to(device)
                targets = torch.tensor(y_valid[j:j+batch_size]).float().to(device)

                outputs = model(inputs)
                v_loss = criterion(outputs, targets)
                valid_loss += v_loss.item()

            valid_loss /= (len(X_valid) / batch_size)

        # Mostrar os resultados de treinamento e validação
        print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {loss.item():.4f}, Validation Loss: {valid_loss:.4f}')

        # Atualizar o melhor modelo se a validação melhorou
        if valid_loss < best_loss:
            best_loss = valid_loss
            epochs_no_improve = 0
            best_model = copy.deepcopy(model.state_dict())
            print(f'New best model saved at epoch {epoch+1} with validation loss {valid_loss:.4f}')
        else:
            epochs_no_improve += 1
            if epochs_no_improve == n_epochs_stop:
                print('Early stopping!')
                break

    return best_model
import torch
import numpy as np
import random

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # for multi-GPU
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = False

seed_everything(42)


In [ ]:
# from sklearn.metrics import cohen_kappa_score
# from sklearn.linear_model import Ridge
# import pickle
# FOLDS = 3
# def comp_score(y_true,y_pred):
#     p = y_pred.clip(1,6).round(0)
#     m = cohen_kappa_score(y_true, p, weights='quadratic')
#     return m
# RESULT_TRAIN = {}
# optimizers = {
#     'Adam': optim.Adam(model.parameters(), lr=0.001),
# #     'SGD': optim.SGD(model.parameters(), lr=0.01, momentum=0.9),
# #     'RMSprop': optim.RMSprop(model.parameters(), lr=0.01),
# #     'AdamW': optim.AdamW(model.parameters(), lr=0.001),
# #     'Adagrad': optim.Adagrad(model.parameters(), lr=0.01),
# #     'Adadelta': optim.Adadelta(model.parameters()),
# #     'Adamax': optim.Adamax(model.parameters(), lr=0.002),
# #     'ASGD': optim.ASGD(model.parameters(), lr=0.01),
# #     'LBFGS': optim.LBFGS(model.parameters(), lr=0.1)
# }


# for n_ in range(3,4,1):
#     for m_ in np.arange(2, 3, 1):
#         for d_ in np.arange(0.2, 0.3, 0.1):
#             for b_ in [32]:
#                 for lr_ in [0.001]:
#                     for lr_step in [3]:
#                         for lr_gamma in [0.5]:
#                             for ac_ in [torch.relu]:
#                                 for opt in optimizers:
#                                     features_selecionadas_lgbm = selecionar_features_importantes([], all_train_embeds,n_)
#                                     all_train_embeds_scaled_new = all_train_embeds_scaled[:, features_selecionadas_lgbm]
#                                     all_test_embeds_scaled_new = all_test_embeds_scaled[:, features_selecionadas_lgbm]
#                                     print('Feature importance retornou esse all_train_embeds', all_train_embeds_scaled_new.shape )
#                                     models_nn=[]
#                                     oof_nn = np.zeros(len(train), dtype='float32')
#                                     test_preds_nn = np.zeros((len(test),FOLDS), dtype='float32')
#                                     for fold in range(FOLDS):
#                                         print('#'*25)
#                                         print('### Fold',fold+1)
#                                         print('#'*25)

#                                         train_index = train["fold"] != fold
#                                         valid_index = train["fold"] == fold

#                                         # Verificar se a GPU está disponível e mover o modelo para a GPU
#                                         device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#                                         print(all_train_embeds_scaled_new.shape)
#                                         X_train = torch.tensor(all_train_embeds_scaled_new[train_index,], dtype=torch.float32).to(device)
#                                         y_train = torch.tensor(train.loc[train_index,'score'].values, dtype=torch.float32).view(-1, 1).to(device)
#                                         X_valid = torch.tensor(all_train_embeds_scaled_new[valid_index,], dtype=torch.float32).to(device)
#                                         y_valid = torch.tensor(train.loc[valid_index,'score'].values, dtype=torch.float32).view(-1, 1).to(device)
#                                         X_test = torch.tensor(all_test_embeds_scaled_new, dtype=torch.float32).to(device)


#                                         # Instantiate the model
#                                         input_size = X_train.shape[1]
#                                         #model = SimpleNN(input_size)
#                                         modelll= DeepNN3
#                                         seed_everything(42)
#                                         model = modelll(input_size, 
#                                                         layer_sizes=[round(1024*m_), round(512*m_), round(256*m_)], 
#                                                         dropout_rate=d_, 
#                                                         use_batch_norm=True,
#                                                         activation_func=ac_
#                                                        )

#                                         model = model.to(device)

#                                         # Define loss function and optimizer
#                                         criterion = nn.MSELoss()
#                                         optimizers = {
#                                             'Adam': optim.Adam(model.parameters(), lr=0.001),
# #                                             'SGD': optim.SGD(model.parameters(), lr=0.01, momentum=0.9),
# #                                             'RMSprop': optim.RMSprop(model.parameters(), lr=0.01),
# #                                             'AdamW': optim.AdamW(model.parameters(), lr=0.001),
# #                                             'Adagrad': optim.Adagrad(model.parameters(), lr=0.01),
# #                                             'Adadelta': optim.Adadelta(model.parameters()),
# #                                             'Adamax': optim.Adamax(model.parameters(), lr=0.002),
# #                                             'ASGD': optim.ASGD(model.parameters(), lr=0.01),
# #                                             'LBFGS': optim.LBFGS(model.parameters(), lr=0.1)
#                                         }
#                                         optimizer = optimizers[opt]

#                                         # Training loop
#                                         best_model_dict = train_model_with_early_stopping(model, criterion, optimizer, X_train, y_train, X_valid, y_valid, 
#                                                                                           num_epochs=30, batch_size=32, n_epochs_stop=2,
#                                                                                           lr_decay_step=lr_step, lr_decay_gamma=lr_gamma)
#                                         seed_everything(42)
#                                         model.load_state_dict(best_model_dict) 

#                                         models_nn.append(model)

#                                         # Evaluate the model
#                                         with torch.no_grad():
#                                             model.eval()
#                                             model = model.to(device)
#                                             outputs = model(X_valid)
#                                             test_loss = criterion(outputs, y_valid)
#                                             print(f'Test Loss: {test_loss.item():.4f}')

#                                         oof_nn[valid_index] = outputs.cpu().numpy().flatten()

#                                         model.eval()
#                                         with torch.no_grad():
#                                             predictions = model(X_test)
#                                         predicted_scores = predictions.cpu().numpy().flatten()
#                                         test_preds_nn[:,fold] = predicted_scores

#                                         score = comp_score(y_valid.cpu().numpy(), outputs.cpu().numpy().flatten())    
#                                         print(f"=> QWK score nn: {score}")
#                                         print()


#                                     print('#'*25)
#                                     score = comp_score(train.score.values, oof_nn)
#                                     print(f'Overall CV QWK score _NN=',score) 
#                                     key = f'{opt}'

#                                     RESULT_TRAIN[key] = {
#                                         'model_name':'NN',
#                                         'test_preds':test_preds_nn,
#                                         'oof':oof_nn,
#                                         'models':models_nn,
#                                         'score':score,
#                                         'features_selecionadas': features_selecionadas_lgbm,

#                                     }

        
# # Salvar RESULT_TRAIN
# with open('RESULT_TRAIN.pkl', 'wb') as f:
#     pickle.dump(RESULT_TRAIN, f)

In [ ]:
# for opt in optimizers:
#     key = f'{opt}'
#     print(f"{key} {RESULT_TRAIN[key]['score']} {sum(RESULT_TRAIN[key]['features_selecionadas'])}")

In [ ]:
from sklearn.metrics import cohen_kappa_score
from sklearn.linear_model import Ridge
import pickle
FOLDS = 3
def comp_score(y_true,y_pred):
    p = y_pred.clip(1,6).round(0)
    m = cohen_kappa_score(y_true, p, weights='quadratic')
    return m

RESULT_TRAIN = {}

MODELS_RANGE = (0,5,1)

for m_ in range(*MODELS_RANGE):
    print(f'{m_}: ----------------------------------------------------')
    # Treina LGBM
    RIDGE_ = False
    LGBM_ = False
    SVR_ = False
    NN_ = False
    if m_==0:
        LGBM_ = True
        models_lgbm=[]
        features_selecionadas_lgbm = selecionar_features_importantes(models_lgbm, all_train_embeds,0)
        all_train_embeds_new = all_train_embeds[:, features_selecionadas_lgbm]
        all_test_embeds_new = all_test_embeds[:, features_selecionadas_lgbm]
        print('Feature importance retornou esse all_train_embeds', all_train_embeds_new.shape )
        models_lgbm=[]
        oof_lgbm = np.zeros(len(train), dtype='float32')
        test_preds_lgbm = np.zeros((len(test),FOLDS), dtype='float32')
    elif m_==1:
        SVR_ = True
        features_selecionadas_lgbm = selecionar_features_importantes([], all_train_embeds,4)
        all_train_embeds_scaled_new = all_train_embeds_scaled[:, features_selecionadas_lgbm]
        all_test_embeds_scaled_new = all_test_embeds_scaled[:, features_selecionadas_lgbm]
        print('Feature importance retornou esse all_train_embeds_scaled', all_train_embeds_scaled_new.shape )
        models_svr=[]
        oof_svr = np.zeros(len(train), dtype='float32')
        test_preds_svr = np.zeros((len(test),FOLDS), dtype='float32')
    #Treina LGBM Feature importance 
    elif m_==2:
        LGBM_ = True
        features_selecionadas_lgbm = selecionar_features_importantes([], all_train_embeds,10)
        all_train_embeds_new = all_train_embeds[:, features_selecionadas_lgbm]
        all_test_embeds_new = all_test_embeds[:, features_selecionadas_lgbm]
        print('Feature importance retornou esse all_train_embeds', all_train_embeds_new.shape )
        models_lgbm=[]
        oof_lgbm = np.zeros(len(train), dtype='float32')
        test_preds_lgbm = np.zeros((len(test),FOLDS), dtype='float32')
    elif m_==3:
        RIDGE_ = True
        features_selecionadas_lgbm = selecionar_features_importantes([], all_train_embeds,10)
        all_train_embeds_scaled_new = all_train_embeds_scaled[:, features_selecionadas_lgbm]
        all_test_embeds_scaled_new = all_test_embeds_scaled[:, features_selecionadas_lgbm]
        print('Feature importance retornou esse all_train_embeds', all_train_embeds_scaled_new.shape )
        models_ridge=[]
        oof_ridge = np.zeros(len(train), dtype='float32')
        test_preds_ridge = np.zeros((len(test),FOLDS), dtype='float32')
    elif m_==4:
        NN_ = True
        features_selecionadas_lgbm = selecionar_features_importantes([], all_train_embeds,3)
        all_train_embeds_scaled_new = all_train_embeds_scaled[:, features_selecionadas_lgbm]
        all_test_embeds_scaled_new = all_test_embeds_scaled[:, features_selecionadas_lgbm]
        print('Feature importance retornou esse all_train_embeds', all_train_embeds_scaled_new.shape )
        models_nn=[]
        oof_nn = np.zeros(len(train), dtype='float32')
        test_preds_nn = np.zeros((len(test),FOLDS), dtype='float32')
   
    for fold in range(FOLDS):
        print('#'*25)
        print('### Fold',fold+1)
        print('#'*25)

        train_index = train["fold"] != fold
        valid_index = train["fold"] == fold
        
        # Modelo LGBM -----------------------------------------------------
        if LGBM_:
            print(all_train_embeds_new.shape)
            X_train = all_train_embeds_new[train_index,]
            y_train = train.loc[train_index,'score'].values
            X_valid = all_train_embeds_new[valid_index,]
            y_valid = train.loc[valid_index,'score'].values
            X_test = all_test_embeds_new


            model = lgb.LGBMRegressor(
                objective=qwk_obj,
                metrics='None',
                learning_rate=0.05,
                max_depth=5,
                num_leaves=10,
                colsample_bytree=0.5,
                reg_alpha=0.1,
                reg_lambda=0.8,
                n_estimators=1500,
                random_state=42,
                extra_trees=True,
                class_weight='balanced',
                verbosity=-1,
                device_type='gpu'
            )

            model.fit(
                X_train, y_train,
                eval_names=['train', 'valid'],
                eval_set=[(X_train, y_train), (X_valid, y_valid)],
                eval_metric=quadratic_weighted_kappa,
                callbacks=callbacks
            )
            models_lgbm.append(model)
            
            preds_lgbm = model.predict(X_valid)
            test_preds_lgbm[:,fold] = model.predict(X_test)
            oof_lgbm[valid_index] = preds_lgbm

            score = comp_score(y_valid, preds_lgbm)    
            print(f"=> QWK score lgbm m_:{m_}: {score}")
            print()
        # Modelo SVR -----------------------------------------
        elif SVR_:
            print(all_train_embeds_scaled_new.shape)
            X_train = all_train_embeds_scaled_new[train_index,]
            y_train = train.loc[train_index,'score'].values
            X_valid = all_train_embeds_scaled_new[valid_index,]
            y_valid = train.loc[valid_index,'score'].values
            X_test = all_test_embeds_scaled_new
            
            model = SVR(C=10) #C=10
            model.fit(X_train, y_train)
            models_svr.append(model)
            
            preds_svr = model.predict(X_valid)
            test_preds_svr[:,fold] = model.predict(X_test)
            oof_svr[valid_index] = preds_svr

            score = comp_score(y_valid, preds_svr)    
            print(f"=> QWK score svr: {score}")
            print()
        elif RIDGE_:
            print(all_train_embeds_scaled_new.shape)
            X_train = all_train_embeds_scaled_new[train_index,]
            y_train = train.loc[train_index,'score'].values
            X_valid = all_train_embeds_scaled_new[valid_index,]
            y_valid = train.loc[valid_index,'score'].values
            X_test = all_test_embeds_scaled_new
            
            model = Ridge(alpha=1.0)
            model.fit(X_train, y_train)
            models_ridge.append(model)
            
            preds_ridge = model.predict(X_valid)
            test_preds_ridge[:,fold] = model.predict(X_test)
            oof_ridge[valid_index] = preds_ridge

            score = comp_score(y_valid, preds_ridge)    
            print(f"=> QWK score svr: {score}")
            print()
        elif NN_:
            # Verificar se a GPU está disponível e mover o modelo para a GPU
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            print(all_train_embeds_scaled_new.shape)
            X_train = torch.tensor(all_train_embeds_scaled_new[train_index,], dtype=torch.float32).to(device)
            y_train = torch.tensor(train.loc[train_index,'score'].values, dtype=torch.float32).view(-1, 1).to(device)
            X_valid = torch.tensor(all_train_embeds_scaled_new[valid_index,], dtype=torch.float32).to(device)
            y_valid = torch.tensor(train.loc[valid_index,'score'].values, dtype=torch.float32).view(-1, 1).to(device)
            X_test = torch.tensor(all_test_embeds_scaled_new, dtype=torch.float32).to(device)
            
            
            # Instantiate the model
            input_size = X_train.shape[1]
            #model = SimpleNN(input_size)
            modelll= DeepNN2
            seed_everything(42)
            model = modelll(
                input_size, 
                            layer_sizes=[2048, 1024, 512], 
                            dropout_rate=0.2, 
                            use_batch_norm=True
            )
            
            model = model.to(device)

            # Define loss function and optimizer
            criterion = nn.MSELoss()
            optimizer = optim.Adam(model.parameters(), lr=0.001)
            
            # Training loop
            num_epochs = 10
            batch_size = 32
            best_model_dict = train_model_with_early_stopping(model, criterion, optimizer, X_train, y_train, X_valid, y_valid, 
                                                              num_epochs=30, batch_size=32, n_epochs_stop=2,
                                                              lr_decay_step=3, lr_decay_gamma=0.5)
            seed_everything(42)
            model.load_state_dict(best_model_dict) 
            
            models_nn.append(model)
            
            # Evaluate the model
            with torch.no_grad():
                model.eval()
                model = model.to(device)
                outputs = model(X_valid)
                test_loss = criterion(outputs, y_valid)
                print(f'Test Loss: {test_loss.item():.4f}')
            
            oof_nn[valid_index] = outputs.cpu().numpy().flatten()
            
            model.eval()
            with torch.no_grad():
                predictions = model(X_test)
            predicted_scores = predictions.cpu().numpy().flatten()
            test_preds_nn[:,fold] = predicted_scores

            score = comp_score(y_valid.cpu().numpy(), outputs.cpu().numpy().flatten())    
            print(f"=> QWK score nn: {score}")
            print()
    
    # Termina fold ---------------------------------------
    if LGBM_:
        RESULT_TRAIN[m_] = {
            'model_name':'LGBM',
            'test_preds':test_preds_lgbm,
            'oof':oof_lgbm,
            'models':models_lgbm,
            'score':score,
            'features_selecionadas': features_selecionadas_lgbm,
            
        }
        # Score LGBM -----------------------------------------------------
        print('#'*25)
        score = comp_score(train.score.values, oof_lgbm)
        print(f'Overall CV QWK score _lgbm=',score)
    if SVR_:
        RESULT_TRAIN[m_] = {
            'model_name':'SVR',
            'test_preds':test_preds_svr,
            'oof':oof_svr,
            'models':models_svr,
            'score':score,
            'features_selecionadas': features_selecionadas_lgbm,
            
        }
        # Score LGBM -----------------------------------------------------
        print('#'*25)
        score = comp_score(train.score.values, oof_svr)
        print(f'Overall CV QWK score _svr=',score    )
    if RIDGE_:
        RESULT_TRAIN[m_] = {
            'model_name':'RIDGE',
            'test_preds':test_preds_ridge,
            'oof':oof_ridge,
            'models':models_ridge,
            'score':score,
            'features_selecionadas': features_selecionadas_lgbm,
            
        }
        # Score LGBM -----------------------------------------------------
        print('#'*25)
        score = comp_score(train.score.values, oof_ridge)
        print(f'Overall CV QWK score _RIDGE=',score)  
    if NN_:
        RESULT_TRAIN[m_] = {
            'model_name':'NN',
            'test_preds':test_preds_nn,
            'oof':oof_nn,
            'models':models_nn,
            'score':score,
            'features_selecionadas': features_selecionadas_lgbm,
            
        }
        # Score LGBM -----------------------------------------------------
        print('#'*25)
        score = comp_score(train.score.values, oof_nn)
        print(f'Overall CV QWK score _NN=',score)  

        
# Salvar RESULT_TRAIN
with open('RESULT_TRAIN.pkl', 'wb') as f:
    pickle.dump(RESULT_TRAIN, f)

In [ ]:
# # Modelo Final Ensamble
# oof = np.zeros(len(train), dtype='float32')
# test_preds = np.zeros(len(test), dtype='float32') 

# # Adiciona os 'oof' dos modelos especificados
# num_models = len(range(*MODELS_RANGE)) +1
# for m_ in range(*MODELS_RANGE):
#     oof += RESULT_TRAIN[m_]['oof']
#     test_preds += np.mean(RESULT_TRAIN[m_]['test_preds'], axis=1)

# oof = (oof 
#        + predicted_train
#       ) / num_models
# test_preds = (test_preds 
#               + predicted_test
#              ) / num_models
# test_preds

# treina ensamble

In [ ]:
# Selecionar as features importantes
features_selecionadas_lgbm = selecionar_features_importantes([], all_train_embeds, 0)

# Filtrar os embeddings com as features selecionadas
all_train_embeds_new = all_train_embeds[:, features_selecionadas_lgbm]
all_test_embeds_new = all_test_embeds[:, features_selecionadas_lgbm]

print('Feature importance retornou esse all_train_embeds', all_train_embeds_new.shape)



# Adiciona os 'oof' dos modelos especificados
for m_ in range(*MODELS_RANGE):
    oof_col = RESULT_TRAIN[m_]['oof'].reshape(-1, 1)
    test_preds_col = np.mean(RESULT_TRAIN[m_]['test_preds'], axis=1).reshape(-1, 1)
    
    # Adicionando colunas aos arrays numpy
    all_train_embeds_new = np.concatenate((all_train_embeds_new, oof_col), axis=1)
    all_test_embeds_new = np.concatenate((all_test_embeds_new, test_preds_col), axis=1)

    
# adiciona deberta
oof_col = predicted_train.to_numpy().reshape(-1, 1)
test_preds_col = predicted_test.reshape(-1, 1)
all_train_embeds_new = np.concatenate((all_train_embeds_new, oof_col), axis=1)
all_test_embeds_new = np.concatenate((all_test_embeds_new, test_preds_col), axis=1)
    
print('Tamanho do all_train_embeds_new:', all_train_embeds_new.shape)
print('Tamanho do all_test_embeds_new:', all_test_embeds_new.shape)

# Mostrar as últimas 7 colunas para visualização
print('Últimas 7 colunas de all_train_embeds_new:')
print(all_train_embeds_new[:, -7:])

print('Últimas 7 colunas de all_test_embeds_new:')
print(all_test_embeds_new[:, -7:])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
            tokenizer=lambda x: x,
            preprocessor=lambda x: x,
            token_pattern=None,
            strip_accents='unicode',
            analyzer = 'word',
            ngram_range=(3,6),
            min_df=0.05,
            max_df=0.95,
            sublinear_tf=True,
)

# Fit e transform no conjunto de treinamento
train_tfid = vectorizer.fit_transform([i for i in train['full_text']])
dense_train_matrix = train_tfid.toarray()

# Transform no conjunto de teste
test_tfid = vectorizer.transform([i for i in test['full_text']])
dense_test_matrix = test_tfid.toarray()

# Concatenar as matrizes TF-IDF com os embeddings fornecidos
all_train_embeds_new = np.concatenate((all_train_embeds_new, dense_train_matrix), axis=1)
all_test_embeds_new = np.concatenate((all_test_embeds_new, dense_test_matrix), axis=1)

# Mostrar as últimas 7 colunas para visualização
print('Últimas 7 colunas de all_train_embeds_new:')
print(all_train_embeds_new.shape)

print('Últimas 7 colunas de all_test_embeds_new:')
print(all_test_embeds_new.shape)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# Definir o vetor CountVectorizer conforme sua especificação
vectorizer_cnt = CountVectorizer(
    tokenizer=lambda x: x,
    preprocessor=lambda x: x,
    token_pattern=None,
    strip_accents='unicode',
    analyzer='word',
    ngram_range=(2, 3),
    min_df=0.10,
    max_df=0.85,
)

# Fit e transform no conjunto de treinamento
train_cnt = vectorizer_cnt.fit_transform([i for i in train['full_text']])
dense_train_matrix_cnt = train_cnt.toarray()

# Transform no conjunto de teste
test_cnt = vectorizer_cnt.transform([i for i in test['full_text']])
dense_test_matrix_cnt = test_cnt.toarray()

# Concatenar as matrizes TF-IDF com os embeddings fornecidos
all_train_embeds_new = np.concatenate((all_train_embeds_new, dense_train_matrix_cnt), axis=1)
all_test_embeds_new = np.concatenate((all_test_embeds_new, dense_test_matrix_cnt), axis=1)

# Mostrar as últimas 7 colunas para visualização
print('Últimas 7 colunas de all_train_embeds_new:')
print(all_train_embeds_new.shape)

print('Últimas 7 colunas de all_test_embeds_new:')
print(all_test_embeds_new.shape)


In [ ]:
models_lgbm=[]
oof_lgbm = np.zeros(len(train), dtype='float32')
test_preds_lgbm = np.zeros((len(test),FOLDS), dtype='float32')

for fold in range(FOLDS):
    print('#'*25)
    print('### Fold',fold+1)
    print('#'*25)

    train_index = train["fold"] != fold
    valid_index = train["fold"] == fold
    
    print(all_train_embeds_new.shape)
    X_train = all_train_embeds_new[train_index,]
    y_train = train.loc[train_index,'score'].values
    X_valid = all_train_embeds_new[valid_index,]
    y_valid = train.loc[valid_index,'score'].values
    X_test = all_test_embeds_new


    model = lgb.LGBMRegressor(
        objective=qwk_obj,
        metrics='None',
        learning_rate=0.05,
        max_depth=5,
        num_leaves=10,
        colsample_bytree=0.5,
        reg_alpha=0.1,
        reg_lambda=0.8,
        n_estimators=1500,
        random_state=42,
        extra_trees=True,
        class_weight='balanced',
        verbosity=-1,
        device_type='gpu'
    )

    model.fit(
        X_train, y_train,
        eval_names=['train', 'valid'],
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        eval_metric=quadratic_weighted_kappa,
        callbacks=callbacks
    )
    models_lgbm.append(model)

    preds_lgbm = model.predict(X_valid)
    test_preds_lgbm[:,fold] = model.predict(X_test)
    oof_lgbm[valid_index] = preds_lgbm

    score = comp_score(y_valid, preds_lgbm)    
    print(f"=> QWK score lgbm m_:{m_}: {score}")
    print()   
    
RESULT_TRAIN[10] = {
    'model_name':'LGBM',
    'test_preds':test_preds_lgbm,
    'oof':oof_lgbm,
    'models':models_lgbm,
    'score':score,
    'features_selecionadas': features_selecionadas_lgbm,

}

In [ ]:
features_selecionadas_lgbm = selecionar_features_importantes(models_lgbm, all_train_embeds_new,5)
all_train_embeds_new2 = all_train_embeds_new[:, features_selecionadas_lgbm]
all_test_embeds_new2 = all_test_embeds_new[:, features_selecionadas_lgbm]
print('Feature importance retornou esse all_train_embeds', all_train_embeds_new2.shape )
models_lgbm=[]
oof_lgbm = np.zeros(len(train), dtype='float32')
test_preds_lgbm = np.zeros((len(test),FOLDS), dtype='float32')

for fold in range(FOLDS):
    print('#'*25)
    print('### Fold',fold+1)
    print('#'*25)

    train_index = train["fold"] != fold
    valid_index = train["fold"] == fold
    
    print(all_train_embeds_new2.shape)
    X_train = all_train_embeds_new2[train_index,]
    y_train = train.loc[train_index,'score'].values
    X_valid = all_train_embeds_new2[valid_index,]
    y_valid = train.loc[valid_index,'score'].values
    X_test = all_test_embeds_new2


    model = lgb.LGBMRegressor(
        objective=qwk_obj,
        metrics='None',
        learning_rate=0.05,
        max_depth=5,
        num_leaves=10,
        colsample_bytree=0.5,
        reg_alpha=0.1,
        reg_lambda=0.8,
        n_estimators=1500,
        random_state=42,
        extra_trees=True,
        class_weight='balanced',
        verbosity=-1,
        device_type='gpu'
    )

    model.fit(
        X_train, y_train,
        eval_names=['train', 'valid'],
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        eval_metric=quadratic_weighted_kappa,
        callbacks=callbacks
    )
    models_lgbm.append(model)

    preds_lgbm = model.predict(X_valid)
    test_preds_lgbm[:,fold] = model.predict(X_test)
    oof_lgbm[valid_index] = preds_lgbm

    score = comp_score(y_valid, preds_lgbm)    
    print(f"=> QWK score lgbm m_:{m_}: {score}")
    print()   
    
RESULT_TRAIN[11] = {
    'model_name':'LGBM',
    'test_preds':test_preds_lgbm,
    'oof':oof_lgbm,
    'models':models_lgbm,
    'score':score,
    'features_selecionadas': features_selecionadas_lgbm,

}



In [ ]:
RESULT_TRAIN[12] = {
    'model_name':'BERT',
    'test_preds':predicted_test,
    'oof':predicted_train,

}

# modelo publico

In [ ]:
def text_prep(x):
    return x
import aes2_preproces_cache_vectorize
TUNE = False

import gc
gc.collect()

import pickle

with open("/kaggle/usr/lib/aes2_added_fb_prize_as_features_preprocessing/train_feats.pickle", "rb") as f:
    train_feats = pickle.load(f)
with open("/kaggle/usr/lib/aes2_added_fb_prize_as_features_preprocessing/X.pickle", "rb") as f:
    X = pickle.load(f)
with open("/kaggle/usr/lib/aes2_added_fb_prize_as_features_preprocessing/y.pickle", "rb") as f:
    y = pickle.load(f)
with open("/kaggle/usr/lib/aes2_added_fb_prize_as_features_preprocessing/y_split.pickle", "rb") as f:
    y_split = pickle.load(f)
with open(
    "/kaggle/input/aes2-eval-added-fb-prize-as-features-8168c5/feature_select.pickle", "rb"
) as f:
    feature_select = pickle.load(f)
    
aes2_preproces_cache_vectorize.feature_select = feature_select

import numpy as np

# X = train_feats[feature_select + ["max_repeated_word_count"]].astype(np.float32).values
X = train_feats[feature_select].astype(np.float32).values
X.shape

from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, cohen_kappa_score
from aes2_preproces_cache_vectorize import *

import builtins
original_print = builtins.print
def patch_print(text, *args, **kwargs):
    if not isinstance(text, str):
        return original_print(text, *args, **kwargs)
    if "	" in text:
        n = text.split("	")[0]
        try:
            n = int(n.strip("[]"))
            if n % 100 == 0:
                original_print(text)
            return None
        except:
            pass
    original_print(text, *args, **kwargs)

builtins.print = patch_print

n_splits = 15
models = []
predictions = []
f1_scores = []
kappa_scores = []


def quadratic_weighted_kappa(y_true, y_pred):
    if isinstance(y_pred, xgb.QuantileDMatrix):
        # XGB
        y_true, y_pred = y_pred, y_true

        y_true = (y_true.get_label() + a).round()
        y_pred = (y_pred + a).clip(1, 6).round()
        qwk = cohen_kappa_score(y_true, y_pred, weights="quadratic")
        return 'QWK', qwk

    else:
        # For lgb
        y_true = y_true + a
        y_pred = (y_pred + a).clip(1, 6).round()
        qwk = cohen_kappa_score(y_true, y_pred, weights="quadratic")
        return 'QWK', qwk, True
    
def qwk_obj(y_true, y_pred):
    labels = y_true + a
    preds = y_pred + a
    preds = preds.clip(1, 6)
    f = 1/2*np.sum((preds-labels)**2)
    g = 1/2*np.sum((preds-a)**2+b)
    df = preds - labels
    dg = preds - a
    grad = (df/g - f*dg/g**2)*len(labels)
    hess = np.ones(len(labels))
    return grad, hess

class Predictor:
    def __init__(self, models: list, n: float = 0.746):
        self.models = models
        self.n = n
#         self.xgb_boost_best_iter = models[1].
    def predict(self, X):
        n_models = len(self.models)
        predicted = None
        n = self.n
        for i, model in enumerate(self.models):
            if i == 0:
                predicted = n*model.predict(X)
            else:
                predicted += (1-n)*model.predict(X)
        return predicted

skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)
xgboost_best_iters = []
light_best_iters = []

_test = pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv")
ENABLE_DONT_WASTE_YOUR_RUN_TIME = len(_test) < 10
# ENABLE_DONT_WASTE_YOUR_RUN_TIME = False
oof = np.zeros(len(train), dtype='float32')
for i, (train_index, test_index) in enumerate(skf.split(X, y_split), 1):
    # Split the data into training and testing sets for this fold
    print('fold',i)
    X_train_fold, X_test_fold = X[train_index], X[test_index]
    y_train_fold, y_test_fold, y_test_fold_int = y[train_index], y[test_index], y_split[test_index]
    callbacks = [log_evaluation(period=25), early_stopping(stopping_rounds=75, first_metric_only=True)]
    light = lgb.LGBMRegressor(
            objective = qwk_obj,
            metrics = 'None',
            learning_rate = 0.05,
            max_depth = 8,
            num_leaves = 10,
            colsample_bytree=0.3,
            reg_alpha = 0.7,
            reg_lambda = 0.1,
            n_estimators=700,
            random_state=42,
            extra_trees=True,
            class_weight='balanced',
            device='gpu' if CUDA_AVAILABLE else 'cpu',
            verbosity = - 1
        )

    # Fit the model on the training data for this fold  
    light.fit(
        X_train_fold,
        y_train_fold,
        eval_names=['train', 'valid'],
        eval_set=[(X_train_fold, y_train_fold), (X_test_fold, y_test_fold)],
        eval_metric=quadratic_weighted_kappa,
        callbacks=callbacks
    )
    light_best_iters.append(light.best_iteration_)
    print(f"Light best iter: {light.best_iteration_}")
   
    xgb_regressor = xgb.XGBRegressor(
        objective = qwk_obj,
        verbosity = 0,
        metrics = 'None',
        learning_rate = 0.1,
        max_depth = 8,
        num_leaves = 10,
        colsample_bytree=0.5,
        reg_alpha = 0.1,
        reg_lambda = 1.6, # validation_1-QWK:0.84010
#         reg_lambda = 0.8, validation_1-QWK:0.83917
        
        n_estimators=1024,
        random_state=42,
#         gamma=0.05,
        extra_trees=True,
        scale_pos_weight=100,
        class_weight='balanced',
        tree_method="hist",
        device="gpu" if CUDA_AVAILABLE else "cpu"
    #             device='gpu',
    #             verbosity = 1
    )
    
    xgb_callbacks = [
        xgb.callback.EvaluationMonitor(period=25),
        xgb.callback.EarlyStopping(75, metric_name="QWK", maximize=True, save_best=True, data_name="validation_1")
    ]
    xgb_regressor.fit(
        X_train_fold,
        y_train_fold,
        eval_set=[(X_train_fold, y_train_fold), (X_test_fold, y_test_fold)],
        eval_metric=quadratic_weighted_kappa,
        callbacks=xgb_callbacks
    )
    print(f"XGBoost best iter: {xgb_regressor.get_booster().best_iteration}")
    xgboost_best_iters.append(xgb_regressor.get_booster().best_iteration)
    predictor = Predictor([light, xgb_regressor], n=0.709)

    models.append(predictor)
    # Make predictions on the test data for this fold
    predictions_fold = predictor.predict(X_test_fold)
    predictions_fold = predictions_fold + a
    oof[test_index] = predictions_fold
    predictions_fold = predictions_fold.clip(1, 6).round()
    predictions.append(predictions_fold)
    # Calculate and store the F1 score for this fold
    f1_fold = f1_score(y_test_fold_int, predictions_fold, average='weighted')
    f1_scores.append(f1_fold)

    # Calculate and store the Cohen's kappa score for this fold
    kappa_fold = cohen_kappa_score(y_test_fold_int, predictions_fold, weights='quadratic')
    kappa_scores.append(kappa_fold)
#         predictor.booster_.save_model(f'fold_{i}.txt')
    cm = confusion_matrix(y_test_fold_int, predictions_fold, labels=[x for x in range(1,7)])

    disp = ConfusionMatrixDisplay(
        confusion_matrix=cm,
        display_labels=[x for x in range(1,7)]
    )
    disp.plot()
    plt.show()
    print(f'F1 score across fold: {f1_fold}')
    print(f'Cohen kappa score across fold: {kappa_fold}')

    gc.collect()
    if ENABLE_DONT_WASTE_YOUR_RUN_TIME:
        break

def find_thresholds(true, pred, steps=50):

    # SAVE TRIALS FOR PLOTTING
    xs = [[],[],[],[],[]]
    ys = [[],[],[],[],[]]

    # COMPUTE BASELINE METRIC
    threshold = [1.5, 2.5, 3.5, 4.5, 5.5]
    pred2 = pd.cut(pred, [-np.inf] + threshold + [np.inf], 
                    labels=[1,2,3,4,5,6]).astype('int32')
    best = cohen_kappa_score(true, pred2, weights="quadratic")

    # FIND FIVE OPTIMAL THRESHOLDS
    for k in range(5):
        for sign in [1,-1]:
            v = threshold[k]
            threshold2 = threshold.copy()
            stop = 0
            while stop<steps:
                # TRY NEW THRESHOLD
                v += sign * 0.01
                threshold2[k] = v
                pred2 = pd.cut(pred, [-np.inf] + threshold2 + [np.inf], 
                                labels=[1,2,3,4,5,6]).astype('int32')
                metric = cohen_kappa_score(true, pred2, weights="quadratic")

                # SAVE TRIALS FOR PLOTTING
                xs[k].append(v)
                ys[k].append(metric)

                # EARLY STOPPING
                if metric<=best:
                    stop += 1
                else:
                    stop = 0
                    best = metric
                    threshold = threshold2.copy()

    # COMPUTE FINAL METRIC
    pred2 = pd.cut(pred, [-np.inf] + threshold + [np.inf], 
                    labels=[1,2,3,4,5,6]).astype('int32')
    best = cohen_kappa_score(true, pred2, weights="quadratic")   

    # RETURN RESULTS
    threshold = [np.round(t,3) for t in threshold]
    return best, threshold, xs, ys

# best, thresholds, xs, ys = find_thresholds(y_split, oof, steps=500)
# print('Best thresholds are:', thresholds )
# print('=> achieve Overall CV QWK score =', best )

thresholds = [1.51, 2.6, 3.5, 4.59, 5.56]

if TUNE:
    import optuna

    def objective(trial):
        xgb_regressor = xgb.XGBRegressor(
            objective = qwk_obj,
            verbosity = 0,
            metrics = 'None',
            learning_rate = 0.1,
            max_depth = 8,
            num_leaves = 10,
            colsample_bytree=0.5,
            reg_alpha = 0.1,
            reg_lambda = trial.suggest_float('reg_lambda', 0, 2, step=0.05),
    #         reg_lambda = 1.0, # validation_1-QWK:0.84010
    #         reg_lambda = 0.8, validation_1-QWK:0.83917

            n_estimators=1024,
            random_state=42,
    #         gamma=0.05,
            extra_trees=True,
            scale_pos_weight=100,
            class_weight='balanced',
            tree_method="hist",
            device="gpu" if CUDA_AVAILABLE else "cpu"
        #             device='gpu',
        #             verbosity = 1
        )

        xgb_callbacks = [
            xgb.callback.EvaluationMonitor(period=25),
            xgb.callback.EarlyStopping(175, metric_name="QWK", maximize=True, save_best=True, data_name="validation_1")
        ]
        predictor = xgb_regressor.fit(
            X_train_fold,
            y_train_fold,
            eval_set=[(X_train_fold, y_train_fold), (X_test_fold, y_test_fold)],
            eval_metric=quadratic_weighted_kappa,
            callbacks=xgb_callbacks
        )

        predictions_fold = predictor.predict(X_test_fold)
        predictions_fold = predictions_fold + a
        predictions_fold = predictions_fold.clip(1, 6).round()
        predictions.append(predictions_fold)
        # Calculate and store the F1 score for this fold


        # Calculate and store the Cohen's kappa score for this fold
        kappa_fold = cohen_kappa_score(y_test_fold_int, predictions_fold, weights='quadratic')
        return kappa_fold

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=100)



    print(f"{study.best_params=}")

    with open("optuna_study.pickle", "wb") as f:
        pickle.dump(study, f)
        
with open("models.pickle", "wb") as f:
    pickle.dump(models, f)

mean_f1_score = np.mean(f1_scores)
mean_kappa_score = np.mean(kappa_scores)
# Print the mean scores
print(f'Mean F1 score across {n_splits} folds: {mean_f1_score}')
print(f'Mean Cohen kappa score across {n_splits} folds: {mean_kappa_score}')
print(f"XGBoost mean best iters: {sum(xgboost_best_iters)/len(xgboost_best_iters)}")
print(f"LightBoost mean best iters: {sum(light_best_iters)/len(light_best_iters)}")

from aes2_preproces_cache_vectorize import preprocess_test, infer

test_feats = preprocess_test()

def infer(test_feats, models):
    probabilities = []
    for model in models:
        proba = model.predict(test_feats) + a
        probabilities.append(proba)
        del model
        gc.collect()

    # Compute the average probabilities across all models
    predictions = np.mean(probabilities, axis=0)
#     predictions = pd.cut(predictions, [-np.inf] + thresholds + [np.inf], 
#                     labels=[1,2,3,4,5,6]).astype('int32')



#     # Print the predictions
#     print(predictions)

#     submission = pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/sample_submission.csv")
#     submission['score'] = predictions
#     submission['score'] = submission['score'].astype(int)
#     submission.to_csv("submission.csv", index=None)
    return predictions


predictions = infer(
    test_feats[feature_select].astype(np.float32),
    models
)
display(predictions)

In [ ]:
display(oof)
display(predictions)

RESULT_TRAIN[13] = {
    'model_name':'public',
    'test_preds':predictions,
    'oof':oof,

}

# ensamble fi

In [ ]:
# Modelo Final Ensamble
oof = np.zeros(len(train), dtype='float32')
test_preds = np.zeros(len(test), dtype='float32') 

MODELS_RANGE = [(13,0.85),(0,0.03),(10,0.02),(11,0.02),(12,0.02),(1,0.02),(4,0.02),(2,0.01),(3,0.01)]
# Adiciona os 'oof' dos modelos especificados
num_models = len(MODELS_RANGE)
for m_, vs_ in MODELS_RANGE:
    oof += (RESULT_TRAIN[m_]['oof'] *vs_)
    # Verifica se 'test_preds' é multidimensional
    if RESULT_TRAIN[m_]['test_preds'].ndim > 1:
        # Se sim, calcula a média ao longo do eixo 1
        test_preds += (np.mean(RESULT_TRAIN[m_]['test_preds'], axis=1) *vs_)
    else:
        # Se não, usa o array como está
        test_preds += (RESULT_TRAIN[m_]['test_preds'] *vs_)

display(oof)
display(test_preds)

# Find QWK Thresholds

In [ ]:
def find_thresholds(true, pred, steps=50):

    # SAVE TRIALS FOR PLOTTING
    xs = [[],[],[],[],[]]
    ys = [[],[],[],[],[]]

    # COMPUTE BASELINE METRIC
    threshold = [1.5, 2.5, 3.5, 4.5, 5.5]
    pred2 = pd.cut(pred, [-np.inf] + threshold + [np.inf], 
                    labels=[1,2,3,4,5,6]).astype('int32')
    best = cohen_kappa_score(true, pred2, weights="quadratic")

    # FIND FIVE OPTIMAL THRESHOLDS
    for k in range(5):
        for sign in [1,-1]:
            v = threshold[k]
            threshold2 = threshold.copy()
            stop = 0
            while stop<steps:

                # TRY NEW THRESHOLD
                v += sign * 0.001
                threshold2[k] = v
                pred2 = pd.cut(pred, [-np.inf] + threshold2 + [np.inf], 
                                labels=[1,2,3,4,5,6]).astype('int32')
                metric = cohen_kappa_score(true, pred2, weights="quadratic")

                # SAVE TRIALS FOR PLOTTING
                xs[k].append(v)
                ys[k].append(metric)

                # EARLY STOPPING
                if metric<=best:
                    stop += 1
                else:
                    stop = 0
                    best = metric
                    threshold = threshold2.copy()

    # COMPUTE FINAL METRIC
    pred2 = pd.cut(pred, [-np.inf] + threshold + [np.inf], 
                    labels=[1,2,3,4,5,6]).astype('int32')
    best = cohen_kappa_score(true, pred2, weights="quadratic")   

    # RETURN RESULTS
    threshold = [np.round(t,3) for t in threshold]
    return best, threshold, xs, ys

In [ ]:
func_ = True
try:
    best, thresholds, xs, ys = find_thresholds(train.score.values, oof, steps=500)
    print('Best thresholds are:', thresholds )
    print('=> achieve Overall CV QWK score =', best )
    func_ = True
except:
    thresholds = [1.5, 2.5, 3.5, 4.5, 5.5]
    xs = [[], [], [], [], []]
    ys = [[], [], [], [], []]
    func_ = False

# Display Threshold Trials

In [ ]:
import matplotlib.pyplot as plt

if func_:

    diff = 0.5
    for k in range(5):
        plt.figure(figsize=(10,3))
        plt.scatter(xs[k],ys[k],s=3)
        m = k+1.5
        plt.xlim((m-diff,m+diff))
        i = np.where( (np.array(xs[k])>m-diff)&(np.array(xs[k])<m+diff) )[0]
        mn = np.min(np.array(ys[k])[i])
        mx = np.max(np.array(ys[k])[i])
        plt.ylim((mn,mx))

        plt.plot([thresholds[k],thresholds[k]],[mn,mx],'--',
                 color='black', label='optimal threshold')

        plt.title(f"Threshold between {k+1} and {k+2}",size=16)
        plt.xlabel('Threshold value',size=10)
        plt.ylabel('QWK CV score',size=10)
        plt.legend()
        plt.show()

# Create Submission CSV

In [ ]:
test_preds = test_preds
print('Test preds shape:', test_preds.shape )
print('First 3 test preds:',test_preds[:3] )

In [ ]:
test_preds_pp = pd.cut(test_preds, [-np.inf] + thresholds + [np.inf], 
                       labels=[1,2,3,4,5,6]).astype('int32')
print('First 3 test preds after PP:',test_preds_pp[:3] )

In [ ]:
sub = pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/sample_submission.csv")
sub["score"] = test_preds_pp
sub.score = sub.score.astype('int32')
sub.to_csv("submission.csv",index=False)
print("Submission shape", sub.shape )
sub.head()